In [37]:
import numpy
from helper import l2_norm, poisson_2d_jacobi, poisson_solution

In [38]:
# Set parameters.
nx = 101  # number of points in the x direction
ny = 101  # number of points in the y direction
xmin, xmax = 0.0, 1.0  # limits in the x direction
ymin, ymax = -0.5, 0.5  # limits in the y direction
Lx = xmax - xmin  # domain length in the x direction
Ly = ymax - ymin  # domain length in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction
dy = Ly / (ny - 1)  # grid spacing in the y direction

# Create the gridline locations and the mesh grid.
x = numpy.linspace(xmin, xmax, num=nx)
y = numpy.linspace(ymin, ymax, num=ny)
X, Y = numpy.meshgrid(x, y)

# Create the source term.
b = (-2 * (numpy.pi / 2)**2 *
     numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly))

# Set the initial conditions.
p0 = numpy.zeros((ny, nx))

# Compute the analytical solution.
p_exact = poisson_solution(x, y, Lx, Ly)

# STEEPEST DESCENT METHOD

In [60]:
def poisson_2d_steepest_descent(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    
    def A(p):
        # Apply the Laplacian operator to p.
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    p = p0.copy()
    rk = numpy.zeros_like(p)  # initial residual
    Ar = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ite = 0  # iteration index
    while diff > rtol and ite < maxiter:
        
        #copy the previous solution
        pk = p.copy()
        
        #compute residual rk
        rk[1:-1, 1:-1] = b[1:-1, 1:-1] - A(pk)
        
        #compute the step size alpha
        Ar[1:-1, 1:-1] = A(rk)
        alpha = numpy.sum(rk * rk) / numpy.sum(rk * Ar)
        
        #update the solution
        p = pk + alpha * rk
        
        #compute relative difference
        diff = l2_norm(p, pk)
        conv.append(diff)
        
        #increment the iteration index
        ite += 1
        
    return p, ite, diff

In [67]:
# Compute the solution using the method of steepest descent.
p, ites, diff = poisson_2d_steepest_descent(p0, b, dx, dy,
                                               maxiter=20000,
                                               rtol=1e-10)
print("iterations = {}".format(ites) + ", diff = {}".format(diff))

iterations = 2, diff = 1.3307695446303778e-16


In [68]:
# Compute the relative L2-norm of the error.
l2_norm(p, p_exact)

0.7499794373094477

# CONJUGATE GRADIENT METHOD

Thus, the full set of steps for the method of conjugate gradients is:

Calculate ${\bf d}^0 = {\bf r}^0$ (just  once), then

1. Calculate $\alpha = \frac{{\bf r}^k \cdot {\bf r}^k}{A{\bf d}^k \cdot {\bf d}^k}$
2. Update ${\bf p}^{k+1}$
3. Calculate ${\bf r}^{k+1} = {\bf r}^k - \alpha A {\bf d}^k$ $\ \ \ \ $(see <a href='#references'>Shewchuk (1994)</a>)
4. Calculate $\beta = \frac{{\bf r}^{k+1} \cdot {\bf r}^{k+1}}{{\bf r}^k \cdot {\bf r}^k}$
5. Calculate ${\bf d}^{k+1}={\bf r}^{k+1}+\beta{\bf d}^{k}$
6. Repeat!

In [76]:
def poisson_2d_conjugate_gradient(p0, b, dx, dy, maxiter=20000, rtol=1e-6):
    
    def A(p):
        # Apply the Laplacian operator to p.
        return (-4.0 * p[1:-1, 1:-1] +
                p[1:-1, :-2] + p[1:-1, 2:] +
                p[:-2, 1:-1] + p[2:, 1:-1]) / dx**2
    
    p = p0.copy()
    r = numpy.zeros_like(p)  # initial residual
    Ad = numpy.zeros_like(p)  # to store the mat-vec multiplication
    conv = []  # convergence history
    diff = rtol + 1  # initial difference
    ite = 0  # iteration index
    
    # Compute the initial residual.
    r[1:-1, 1:-1] = b[1:-1, 1:-1] - A(p)
    
    # Set the initial search direction to be the residual.
    d = r.copy()
    
    while diff > rtol and ite < maxiter:
        
        #copy the initial solution
        pk = p.copy()
        rk = r.copy()
        
        #compute the step size alpha
        Ad[1:-1, 1:-1] = A(d)
        alpha = numpy.sum(r * r) / numpy.sum(d * Ad)
      
        #update the solution
        p = pk + alpha * d
        r = rk - alpha * Ad
       
        #calculate beta parameter
        beta = numpy.sum(r * r) / numpy.sum(rk * rk)
        
        #update the search direction vector d
        d = r + beta * d
        
        #compute the difference
        diff = l2_norm(p, pk)
        conv.append(diff)
        
        #increment the iteration
        ite += 1
        
    return p, ite, diff

In [78]:
# Compute the solution using the method of conjugate gradients.
p, ites, diff = poisson_2d_conjugate_gradient(p0, b, dx, dy,
                                                 maxiter=20000,
                                                 rtol=1e-10)
print("iterations = {}".format(ites) + ", diff = {}".format(diff))

iterations = 2, diff = 1.2982770796281907e-16


In [79]:
# Compute the relative L2-norm of the error.
l2_norm(p, p_exact)

0.7499794373094477

In [80]:
# Compute the solution using Jacobi relaxation.
p, ites, conv_jacobi = poisson_2d_jacobi(p0, b, dx, dy,
                                         maxiter=40000,
                                         rtol=1e-10)
print('Jacobi relaxation: {} iterations '.format(ites) +
      'to reach a relative difference of {}'.format(conv_jacobi[-1]))

Jacobi relaxation: 31227 iterations to reach a relative difference of 9.997923503623598e-11


# MORE DIFFICULT POISSON PROBLEMS

In [83]:
# Modify the source term of the Poisson system.
b2 = (numpy.sin(numpy.pi * X / Lx) *
     numpy.cos(numpy.pi * Y / Ly) +
     numpy.sin(6.0 * numpy.pi * X / Lx) *
     numpy.sin(6.0 * numpy.pi * Y / Ly))

In [84]:
maxiter, rtol = 40000, 1e-10
p, ites, conv = poisson_2d_jacobi(p0, b2, dx, dy,
                                  maxiter=maxiter, rtol=rtol)
print('Jacobi relaxation: {} iterations'.format(ites))
p, ites, conv = poisson_2d_steepest_descent(p0, b2, dx, dy,
                                            maxiter=maxiter,
                                            rtol=rtol)
print('Method of steepest descent: {} iterations'.format(ites))
p, ites, conv = poisson_2d_conjugate_gradient(p0, b2, dx, dy,
                                              maxiter=maxiter,
                                              rtol=rtol)
print('Method of conjugate gradients: {} iterations'.format(ites))

Jacobi relaxation: 31226 iterations
Method of steepest descent: 27059 iterations
Method of conjugate gradients: 3 iterations
